In [2]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import heapq

mp_pose = mp.solutions.pose

def detectPose(image, pose, display=True):
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Perform pose detection
    results = pose.process(image_rgb)
    landmarks = None
    if results.pose_landmarks:
        landmarks = results.pose_landmarks
        if display:
            mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return image, landmarks

def detecte_position(video_dir='videos/video', output_dir='output_frame', top_n=5):
    # Initialize MediaPipe Pose model
    pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize the list to store data for the Excel file
    data = []

    # Process each video in the directory
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".mp4"):
            video_path = os.path.join(video_dir, video_filename)
            video = cv2.VideoCapture(video_path)

            if not video.isOpened():
                print(f"Error: Video file {video_filename} could not be opened.")
                continue

            top_frames = []  # Use a heap to store the top frames with the maximum z values

            while video.isOpened():
                ok, frame = video.read()

                if not ok:
                    break

                frame = cv2.flip(frame, 1)  # Flip frame horizontally
                frame_height, frame_width, _ = frame.shape

                frame, landmarks = detectPose(frame, pose_video, display=False)
                if landmarks:
                    # Extract coordinates of point 19
                    point19 = landmarks.landmark[19]
                    x19 = point19.x * frame_width
                    y19 = point19.y * frame_height
                    z19 = point19.z

                    # Extract coordinates of point 20
                    point20 = landmarks.landmark[20]
                    x20 = point20.x * frame_width
                    y20 = point20.y * frame_height
                    z20 = point20.z

                    # Determine the point with the maximum z value
                    if z19 > z20:
                        heapq.heappush(top_frames, (z19, frame.copy(), x19, y19, z19, '19'))
                    else:
                        heapq.heappush(top_frames, (z20, frame.copy(), x20, y20, z20, '20'))

                    # Keep only the top frames with the maximum z values
                    if len(top_frames) > top_n:
                        heapq.heappop(top_frames)

            video.release()

            # Save the top frames with the maximum z values and their coordinates
            if top_frames:
                top_frames_sorted = sorted(top_frames, reverse=True)
                x_coords = [frame_info[2] for frame_info in top_frames_sorted]
                y_coords = [frame_info[3] for frame_info in top_frames_sorted]
                z_coords = [frame_info[4] for frame_info in top_frames_sorted]
                frame_paths = []
                for i, (z, frame, x, y, z_value, point) in enumerate(top_frames_sorted):
                    frame_filename = f"{video_filename}_frame_max_z_{i+1}.png"
                    max_z_frame_path = os.path.join(output_dir, frame_filename)
                    #cv2.imwrite(max_z_frame_path, frame)
                    frame_paths.append(max_z_frame_path)

                data.append({
                    'video_path': video_path,
                    'frame_paths': frame_paths,
                    'x_coords': x_coords,
                    'y_coords': y_coords,
                    'z_coords': z_coords
                })

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    excel_path = os.path.join(output_dir, 'video_frames_info.xlsx')
    df.to_excel(excel_path, index=False)

    print(f"Data saved to {excel_path}")




In [5]:
# Example usage
detecte_position()

Data saved to output_frame22\video_frames_info.xlsx
